In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from IPython.display import clear_output

from collections import Counter

import nltk

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

import gensim
# Instrucciones: https://radimrehurek.com/gensim/install.html
# Yo baje el 'gensim-3.4.0.tar.gz' de  'https://pypi.org/project/gensim/#files' y se instala facil tirando terminal desde el directorio extraido y  'python setup.py install'
# Restar Kernel y sale andando
from gensim import corpora
from gensim.models.word2vec import Word2Vec

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rodrigo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rodrigo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preparo y entreno el label encoder y el one hot encoder con la info de todos los avisos existentes

In [2]:
avisos_totales = pd.concat([pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/Fiuba desde 15 Abril/fiuba_6_avisos_detalle.csv', usecols=['idaviso', 'nombre_area']), pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv', usecols=['idaviso', 'nombre_area'])])
avisos_totales.drop_duplicates(subset=['idaviso'], inplace=True)
avisos_totales.reset_index(drop=True, inplace=True)
avisos_totales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25288 entries, 0 to 25287
Data columns (total 2 columns):
idaviso        25288 non-null int64
nombre_area    25288 non-null object
dtypes: int64(1), object(1)
memory usage: 395.2+ KB


In [3]:
avisos_totales['nombre_area'].value_counts().size

188

- Label encoding para las 188 areas, y entreno el One Hot Encoding con esos 188 codigos (0-187)

In [4]:
#Fundamental usar para el label encoder una columna DF que contenga TODOS los posibles valores categoricos del nombre de area, es decir, lo saco del DF de TODOS los anuncios.
#Esto es para que si en un set no estan todos, igual los codifique de la misma forma. SI lo entreno con un set que no tiene todos, al ingresar un set con valores nuevos
#va a tirar error porque no tenia esos labels entrenados

#UNA VEZ ENTRENADO EL LABEL ENCODER, Y EL ONEHOTENCODER, USAR SIEMRE EL MISMO PARA EL RESTO DEL NOTEBOOK CODIFICAR LOS SETS DE DATA (TEST Y TRAIN) Y EL SET DE TEST100K

le = preprocessing.LabelEncoder()
avisos_totales['codigo_area'] = le.fit_transform(avisos_totales['nombre_area'].astype(str))

#PARA EL ONE HOT ENCODING, AVISARLE QUE LA CANTIDAD DE VALORES CATEGORICOS ES EL DE TODAS LAS CATEGORIAS EN EL DF DE TODOS LOS ANUNCIOS:188
enc = preprocessing.OneHotEncoder(188)   
enc.fit(avisos_totales['codigo_area'].values.reshape(-1,1))

avisos_totales.head()

,idaviso,nombre_area,codigo_area
0,8725750,Comercial,30
1,17903700,Salud,158
2,1000150677,Transporte,181
3,1000610287,Transporte,181
4,1000872556,Producción,143


### Cargo historial de postulaciones x usuario x area, ya encodeado con One Hot Encoding

In [5]:
users_ids_history = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/historial_cant_post_xuser_xarea.csv', dtype={i+1:'int16' for i in range(188)})
users_ids_history.rename(columns={col:('h'+col) for col in users_ids_history.columns[1:]}, inplace=True)
users_ids_history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346764 entries, 0 to 346763
Columns: 189 entries, idpostulante to h187
dtypes: int16(188), object(1)
memory usage: 127.0+ MB


### Cargo Dataset y test100k para operar

In [6]:
data_set_complete_v2 = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/data_set_complete_v2.csv', usecols=['sepostulo','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente','post_x_aviso','vistas_x_aviso'],\
                                dtype={'sepostulo':'int8', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8', 'post_x_aviso':'int32','vistas_x_aviso':'int32'})

data_set_complete_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 21 columns):
sepostulo           int8
idpostulante        object
veces_visto         int16
sexo                int8
edad                int8
titulo              object
nombre_area         category
estudio_encurso     int8
estudio_graduado    int8
tipo_full-time      int8
tipo_part-time      int8
tipo_otro           int8
zona_gba            int8
zona_caba           int8
nivel_senior        int8
nivel_junior        int8
nivel_otro          int8
nivel_jefe          int8
nivel_gerente       int8
post_x_aviso        int32
vistas_x_aviso      int32
dtypes: category(1), int16(1), int32(2), int8(15), object(2)
memory usage: 123.0+ MB


In [7]:
#Achico a 1 millon para maniabrabilidad en terminos de performance
data = data_set_complete_v2.sample(1000000)

del data_set_complete_v2

data.reset_index(drop=True, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 21 columns):
sepostulo           1000000 non-null int8
idpostulante        1000000 non-null object
veces_visto         1000000 non-null int16
sexo                1000000 non-null int8
edad                1000000 non-null int8
titulo              1000000 non-null object
nombre_area         1000000 non-null category
estudio_encurso     1000000 non-null int8
estudio_graduado    1000000 non-null int8
tipo_full-time      1000000 non-null int8
tipo_part-time      1000000 non-null int8
tipo_otro           1000000 non-null int8
zona_gba            1000000 non-null int8
zona_caba           1000000 non-null int8
nivel_senior        1000000 non-null int8
nivel_junior        1000000 non-null int8
nivel_otro          1000000 non-null int8
nivel_jefe          1000000 non-null int8
nivel_gerente       1000000 non-null int8
post_x_aviso        1000000 non-null int32
vistas_x_aviso      1000000 non-null i

In [8]:
test100k = pd.read_csv('/home/rodrigo/ENV/Notebooks/Tp2/Datos-Tp2/Csvs/test100k_complete_v2.csv', usecols=['id','idpostulante','veces_visto','sexo','edad','titulo','nombre_area',\
                                                                  'estudio_encurso','estudio_graduado','tipo_full-time','tipo_part-time','tipo_otro',\
                                                                  'zona_gba','zona_caba','nivel_senior','nivel_junior','nivel_otro','nivel_jefe','nivel_gerente','post_x_aviso','vistas_x_aviso'],\
                                dtype={'id':'int64', 'idpostulante':'object', 'veces_visto':'int16', 'sexo':'int8', 'edad':'int8', 'nombre_area':'category',\
                                       'estudio_encurso':'int8', 'estudio_graduado':'int8', 'tipo_full-time':'int8','tipo_part-time':'int8','tipo_otro':'int8',\
                                       'zona_gba':'int8','zona_caba':'int8','nivel_senior':'int8','nivel_junior':'int8','nivel_otro':'int8','nivel_jefe':'int8','nivel_gerente':'int8','post_x_aviso':'int32','vistas_x_aviso':'int32'})

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 21 columns):
id                  100000 non-null int64
idpostulante        100000 non-null object
veces_visto         100000 non-null int16
sexo                100000 non-null int8
edad                100000 non-null int8
titulo              100000 non-null object
nombre_area         100000 non-null category
estudio_encurso     100000 non-null int8
estudio_graduado    100000 non-null int8
tipo_full-time      100000 non-null int8
tipo_part-time      100000 non-null int8
tipo_otro           100000 non-null int8
zona_gba            100000 non-null int8
zona_caba           100000 non-null int8
nivel_senior        100000 non-null int8
nivel_junior        100000 non-null int8
nivel_otro          100000 non-null int8
nivel_jefe          100000 non-null int8
nivel_gerente       100000 non-null int8
post_x_aviso        100000 non-null int32
vistas_x_aviso      100000 non-null int32
dtypes: category(

### Preprocessing del titulo

In [9]:
# Palabras del tipo para, con, el, de, y, sin, la, que, en, ... etc. 
stop = stopwords.words('spanish')

# Stemmer, trata de convertir palabras derivadas de una palabra, a la palabra raiz
stemmer = SnowballStemmer("spanish")

- Para data

In [10]:
data['titulo'].head()

0                 Recepcionista
1    Auxiliar Técnico Part Time
2                 Recepcionista
3          Analista Contable Jr
4     Asistente de Arquitectura
Name: titulo, dtype: object

In [11]:
#Remuevo esos caracteres
data['titulo_processed'] = data['titulo'].str.replace('/', ' ').str.replace('-', ' ').str.replace('(', ' ').str.replace(')', ' ')
#Codifico de esta forma y paso caracteres con tildes u otras cosas raras a caracteres base
data['titulo_processed'] = data['titulo_processed'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#Remuevo puntuaciones
data['titulo_processed'] = data['titulo_processed'].str.replace('[^\w\s]',' ')
#Processing
data['titulo_processed'] = data['titulo_processed'].apply(lambda x: ' '.join(stemmer.stem(x.lower()) for x in x.split() if ((not x.isdigit()) and (x not in stop) and (len(x)>2))))  

data['titulo_processed'].head()

0                 recepcion
1    auxili tecnic part tim
2                 recepcion
3              anal contabl
4      asistent arquitectur
Name: titulo_processed, dtype: object

- Para test100k

In [12]:
test100k['titulo'].head()

0                            Asistente Comercial
1                            Asistente Comercial
2                            Asistente Comercial
3    Oracle Financials Sr. Application Developer
4    Oracle Financials Sr. Application Developer
Name: titulo, dtype: object

In [13]:
#Remuevo esos caracteres
test100k['titulo_processed'] = test100k['titulo'].str.replace('/', ' ').str.replace('-', ' ').str.replace('(', ' ').str.replace(')', ' ')
#Codifico de esta forma y paso caracteres con tildes u otras cosas raras a caracter base
test100k['titulo_processed'] = test100k['titulo_processed'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#Remuevo puntuaciones
test100k['titulo_processed'] = test100k['titulo_processed'].str.replace('[^\w\s]',' ')
#Processing
test100k['titulo_processed'] = test100k['titulo_processed'].apply(lambda x: ' '.join(stemmer.stem(x.lower()) for x in x.split() if ((not x.isdigit()) and (x not in stop) and (len(x)>2))))  

test100k['titulo_processed'].head()

0                      asistent comercial
1                      asistent comercial
2                      asistent comercial
3    oracl financials application develop
4    oracl financials application develop
Name: titulo_processed, dtype: object

- Saco palabras de poca frecuencia

In [14]:
low_freq_data = pd.Series(' '.join(data['titulo_processed']).split()).value_counts().to_frame()
low_freq_test100k = pd.Series(' '.join(test100k['titulo_processed']).split()).value_counts().to_frame()
low_freq = pd.concat([low_freq_data, low_freq_test100k]).reset_index().groupby(by='index').sum()
low_freq = low_freq[low_freq[0]<300]
low_freq = list(low_freq.index)
len(low_freq)

3275

In [15]:
data['titulo_processed'] = data['titulo_processed'].apply(lambda x: ' '.join(set(x.split()) - set(low_freq)))
                                                          
test100k['titulo_processed'] = test100k['titulo_processed'].apply(lambda x: ' '.join(set(x.split()) - set(low_freq)))

del low_freq

### Uso label encoding para nombre_area

In [16]:
data['nombre_area'] = (le.transform(data['nombre_area'].astype(str))).astype('int16')

In [17]:
test100k['nombre_area'] = (le.transform(test100k['nombre_area'].astype(str))).astype('int16')

### One Hot Encoding para nombre_area

In [18]:
#encoded_areas = enc.transform(data['nombre_area'].values.reshape(-1,1))
#encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

#data = data.merge(encoded_areas_df, left_index=True, right_index=True)

#del encoded_areas, encoded_areas_df

data = data.merge((pd.DataFrame((enc.transform(data['nombre_area'].values.reshape(-1,1))).toarray()).astype('int8').rename(columns={col:('a'+str(col)) for col in list(range(188))})), left_index=True, right_index=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 210 entries, sepostulo to a187
dtypes: int16(2), int32(2), int8(203), object(3)
memory usage: 227.9+ MB


In [19]:
data.head(2)

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,a178,a179,a180,a181,a182,a183,a184,a185,a186,a187
0,0,Nzd4qNB,0,0,39,Recepcionista,153,4,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,pzNQ48b,0,1,27,Auxiliar Técnico Part Time,168,0,4,1,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#encoded_areas = enc.transform(test100k['nombre_area'].values.reshape(-1,1))
#encoded_areas_df = pd.DataFrame(encoded_areas.toarray()).astype('int8')

#test100k = test100k.merge(encoded_areas_df, left_index=True, right_index=True)

#del encoded_areas, encoded_areas_df

test100k = test100k.merge((pd.DataFrame((enc.transform(test100k['nombre_area'].values.reshape(-1,1))).toarray()).astype('int8').rename(columns={col:('a'+str(col)) for col in list(range(188))})), left_index=True, right_index=True)

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 210 entries, id to a187
dtypes: int16(2), int32(2), int64(1), int8(202), object(3)
memory usage: 23.5+ MB


In [21]:
test100k.head(2)

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,a178,a179,a180,a181,a182,a183,a184,a185,a186,a187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,0,0,0


### Merge del historial con One Hot Coding

In [22]:
#Aca meto la info del historial (matriz de 188 con el one hot encoding)
data = data.merge((data['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int16')), left_index=True, right_index=True)

#Estandarizacion (no dio buenos resultados)
#data = data.merge(pd.DataFrame(preprocessing.scale(data['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0))).astype('int16'), left_index=True, right_index=True)


data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 398 entries, sepostulo to h187
dtypes: int16(190), int32(2), int8(203), object(3)
memory usage: 586.5+ MB


In [23]:
data.head(2)

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,h178,h179,h180,h181,h182,h183,h184,h185,h186,h187
0,0,Nzd4qNB,0,0,39,Recepcionista,153,4,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,pzNQ48b,0,1,27,Auxiliar Técnico Part Time,168,0,4,1,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#Aca meto la info del historial (matriz de 188 con el one hot encoding)
test100k = test100k.merge((test100k['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0).astype('int16')), left_index=True, right_index=True)

#Estandarizacion (no dio buenos resultados)
#test100k = test100k.merge(pd.DataFrame(preprocessing.scale(test100k['idpostulante'].to_frame().merge(users_ids_history, how='left', on='idpostulante').drop(columns='idpostulante').fillna(value=0))).astype('int16'), left_index=True, right_index=True)

test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 398 entries, id to h187
dtypes: int16(190), int32(2), int64(1), int8(202), object(3)
memory usage: 59.3+ MB


In [25]:
test100k.head(2)

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,h178,h179,h180,h181,h182,h183,h184,h185,h186,h187
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,0,0,0,0,0,0,0,0,0,0
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,0,0,0,0,0,0,0,4,0,0


### Vectorizing del titulo

- Entreno el modelo que me va a transformar palabras a vectores. Lo entreno con todas las palabras de titulo_processed de ambos DF, data y test100k

In [26]:
#Armo lista de tokens para entrenar modelo que va a pasar palabras a vectores
token_list = [nltk.word_tokenize(x) for x in data['titulo_processed'].values] + [nltk.word_tokenize(y) for y in test100k['titulo_processed'].values]
# train model
model = Word2Vec(token_list, min_count=1, workers=6)
# save model
model.save('model.bin')

- Lo armo para Data

In [27]:
#Me genero una estructura matriz de 100 dimensiones, que es la dimension de los vectores que me va a entrengar Word2Vec model
encoded_text = pd.DataFrame(np.zeros((data.shape[0], 100)))

#Aca lo que hago es agarrar cada titulo procesado, calculo el vector para cada palabra que tiene, y luego hago el promedio de los vectores de todas las palabras en el titulo
i=0
vec=0
count=0
total_size=data.shape[0]

if(data.columns[21]=='titulo_processed'):
    
    while(i<total_size):

        vec=0
        count=1

        for word in data.iat[i,21].split():
            vec = vec + model.wv.__getitem__(word)
            count += 1

        if(count!=0):
            vec = vec / count
            encoded_text.iloc[i,:] = vec

        i+=1
else:
    print('Ver que numero de columna es titulo_processed en data')

In [28]:
#normalizo
encoded_text = pd.DataFrame(normalize(encoded_text)).astype('float32').rename(columns={col:('t'+str(col)) for col in list(range(100))})

data = pd.concat([data,encoded_text], axis=1)

del encoded_text

data.head(2)

,sepostulo,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,t90,t91,t92,t93,t94,t95,t96,t97,t98,t99
0,0,Nzd4qNB,0,0,39,Recepcionista,153,4,0,1,...,-0.068614,-0.130280,-0.029324,0.053977,-0.047982,0.055445,0.030157,0.079307,-0.186681,0.124011
1,0,pzNQ48b,0,1,27,Auxiliar Técnico Part Time,168,0,4,1,...,-0.080211,-0.087083,-0.074131,-0.021384,-0.073854,-0.091832,-0.110122,-0.085452,0.082681,0.090163


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Columns: 498 entries, sepostulo to t99
dtypes: float32(100), int16(190), int32(2), int8(203), object(3)
memory usage: 968.0+ MB


- Lo armo para Test100k

In [30]:
#Me genero una estructura matriz de 100 dimensiones, que es la dimension de los vectores que me va a entrengar Word2Vec model
encoded_text = pd.DataFrame(np.zeros((test100k.shape[0], 100)))

#Aca lo que hago es agarrar cada titulo procesado, calculo el vector para cada palabra que tiene, y luego hago el promedio de los vectores de todas las palabras en el titulo
i=0
vec=0
count=0
total_size=test100k.shape[0]

if(test100k.columns[21]=='titulo_processed'):
    
    while(i<total_size):

        vec=0
        count=0

        for word in test100k.iat[i,21].split():
            vec = vec + model.wv.__getitem__(word)
            count += 1

        if(count!=0):
            vec = vec / count
            encoded_text.iloc[i,:] = vec

        i+=1
else:
    print('Ver que numero de columna es titulo_processed en test100k')

In [31]:
#normalizo
encoded_text = pd.DataFrame(normalize(encoded_text)).astype('float32').rename(columns={col:('t'+str(col)) for col in list(range(100))})

test100k = pd.concat([test100k,encoded_text], axis=1)

del encoded_text

test100k.head(2)

,id,idpostulante,veces_visto,sexo,edad,titulo,nombre_area,estudio_encurso,estudio_graduado,tipo_full-time,...,t90,t91,t92,t93,t94,t95,t96,t97,t98,t99
0,0,6M9ZQR,0,0,42,Asistente Comercial,30,0,7,1,...,-0.069899,-0.169342,-0.085861,-0.058281,0.157024,0.08219,-0.029177,0.040164,0.149889,-0.102072
1,1,6v1xdL,0,1,30,Asistente Comercial,30,2,3,1,...,-0.069899,-0.169342,-0.085861,-0.058281,0.157024,0.08219,-0.029177,0.040164,0.149889,-0.102072


In [32]:
test100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 498 entries, id to t99
dtypes: float32(100), int16(190), int32(2), int64(1), int8(202), object(3)
memory usage: 97.5+ MB


### Pruebo con Random Forest

In [33]:
#Saco edad porque vimos que resta score
#del train_set, test_set, train_classes, test_classes
train_set, test_set, train_classes, test_classes = train_test_split(data.drop(columns=['edad', 'sepostulo', 'idpostulante', 'titulo', 'titulo_processed']), data['sepostulo'], test_size=0.3)

In [34]:
clf = RandomForestClassifier(n_estimators=60, min_samples_leaf=10, min_samples_split=5, n_jobs=-1).fit(train_set, train_classes)
print('Test:', clf.score(test_set, test_classes), 'Entrenamiento:', clf.score(train_set, train_classes))

MemoryError: 

In [ ]:
test100k['sepostulo'] = clf.predict(test100k.drop(columns=['edad', 'id', 'idpostulante', 'titulo', 'titulo_processed']))
test100k[['id','sepostulo']].to_csv('Resultados/n60_minl10_mins5.csv', index=False)

- Con Normalization, stemming, low_freq 100
    - Con Standardization de historial
        - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
            - 0.8558233333333334 Entrenamiento: 0.8896028571428571
    - Sin Standardization de historial
        - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
            - Test: 0.9083366666666667 Entrenamiento: 0.944 

- Doc2Vec Segundo intento, con stemming.
    - Sin normalizacion
        - Con filtro low_freq 100
            - (n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
                - Test: 0.9106366666666666 Entrenamiento: 0.9535442857142857
            - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
                - Test: 0.9075733333333333 Entrenamiento: 0.9435157142857142
    - Con normalizacion
        - Con filtro low_freq 100
            - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
                - Test: 0.9083366666666667 Entrenamiento: 0.944
            - Sin stemming:
                - Test: 0.9068966666666667 Entrenamiento: 0.9434057142857143
        - Con filtro low_freq 20
            - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
                - 0.9080766666666666 Entrenamiento: 0.9439728571428572
        - Con filtro low_freq 300
            - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
                - Test: 0.9078933333333333 Entrenamiento: 0.9435942857142857
        - Con filtro low_freq 1000
            - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
                - Test: 0.9076933333333334 Entrenamiento: 0.9428942857142857

- Doc2Vec Primer intento, filtro low_freq separados de 20/5 para data/test100k respectivamente
    - (n_estimators=30, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
        - Test: 0.9112666666666667 Entrenamiento: 0.9538957142857143
    - (n_estimators=30, min_samples_leaf=8, min_samples_split=4, n_jobs=-1)
        - Test: 0.9103733333333334 Entrenamiento: 0.9507428571428571
    - (n_estimators=30, min_samples_leaf=10, min_samples_split=4, n_jobs=-1)
        - Test: 0.90886 Entrenamiento: 0.9441471428571429
    - (n_estimators=30, min_samples_leaf=10, min_samples_split=5, n_jobs=-1)
        - Test: 0.9077866666666666 Entrenamiento: 0.9438542857142858
    - (n_estimators=30, min_samples_leaf=7, min_samples_split=6, n_jobs=-1)
        - Test: 0.9103166666666667 Entrenamiento: 0.9541357142857143
    - (n_estimators=60, min_samples_leaf=7, min_samples_split=4, n_jobs=-1)
        - Comun
            - Test: 0.91227 Entrenamiento: 0.9548971428571429

### Busco el parametro optimo con GridSearch

In [ ]:
del data

- Pruebo distinta cantidad de arboles

#GridSearch, busco los mejores parametros
parameters = {'n_estimators':[4, 8, 12, 16, 32, 64, 100, 128]}
rndmtree = RandomForestClassifier(n_jobs=-1)
rndmtreeGS = GridSearchCV(rndmtree, parameters)
rndmtreeGS.fit(train_set, train_classes)
rndmtreeGS.best_params_
#128

print('Test:', rndmtreeGS.score(test_set, test_classes), 'Entrenamiento:', rndmtreeGS.score(train_set, train_classes))

rndmtreeGS.cv_results_

- {'mean_fit_time': array([ 10.65733536,  18.0809954 ,  21.97569235,  28.99000303,
         53.74215492, 105.21326248, 165.86294913, 200.48018638]),
 'mean_score_time': array([1.22243245, 1.42882133, 1.56605172, 1.70461313, 2.45767085,
        4.24544422, 6.11957598, 7.0762221 ]),
 'mean_test_score': array([0.85763714, 0.88921143, 0.89835143, 0.90388714, 0.91139143,
        0.91462143, 0.91596429, 0.91653714]),
 'mean_train_score': array([0.97680929, 0.994175  , 0.99758643, 0.99872   , 0.99975   ,
        0.99994071, 0.99995357, 0.99995429]),
 'param_n_estimators': masked_array(data=[4, 8, 12, 16, 32, 64, 100, 128],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 4},
  {'n_estimators': 8},
  {'n_estimators': 12},
  {'n_estimators': 16},
  {'n_estimators': 32},
  {'n_estimators': 64},
  {'n_estimators': 100},
  {'n_estimators': 128}],
 'rank_test_score': array([8, 7, 6, 5, 4, 3, 2, 1], dtype=int32),
 'split0_test_score': array([0.85991326, 0.88871318, 0.89882743, 0.9045617 , 0.91193311,
        0.91495881, 0.91656595, 0.91700738]),
 'split0_train_score': array([0.9768764 , 0.99425713, 0.99749285, 0.99868643, 0.99976429,
        0.99994   , 0.99995071, 0.99995071]),
 'split1_test_score': array([0.8577398 , 0.89007984, 0.89808985, 0.90435986, 0.9104413 ,
        0.91443559, 0.91535702, 0.91634702]),
 'split1_train_score': array([0.9768593 , 0.99422072, 0.99760429, 0.99873357, 0.99973   ,
        0.99994429, 0.999955  , 0.99995714]),
 'split2_test_score': array([0.85525836, 0.88884127, 0.898137  , 0.90273986, 0.91179987,
        0.91446988, 0.91596988, 0.91625702]),
 'split2_train_score': array([0.97669216, 0.99404715, 0.99766214, 0.99874   , 0.99975571,
        0.99993786, 0.999955  , 0.999955  ]),
 'std_fit_time': array([0.26896197, 0.31426464, 0.20138819, 0.45966349, 0.57356212,
        3.9532968 , 1.7151291 , 5.1175697 ]),
 'std_score_time': array([0.01616008, 0.02108894, 0.06959461, 0.00881401, 0.05376479,
        0.19489816, 0.1942703 , 0.0453662 ]),
 'std_test_score': array([0.00190174, 0.00061628, 0.00033714, 0.00081542, 0.00067404,
        0.00023898, 0.00049356, 0.00033453]),
 'std_train_score': array([8.31142393e-05, 9.16199328e-05, 7.02569739e-05, 2.38845211e-05,
        1.45684517e-05, 2.67262196e-06, 2.02035343e-06, 2.67265755e-06])}

- Pruebo distinta cantidad de muestras necesarias para cada split

#GridSearch, busco los mejores parametros
parameters = {'min_samples_split':[3, 6, 10, 0.05, 0.1, 0.3, 0.5]}
rndmtree = RandomForestClassifier(n_estimators=20, n_jobs=-1)
rndmtreeGS = GridSearchCV(rndmtree, parameters)
rndmtreeGS.fit(train_set, train_classes)
rndmtreeGS.best_params_
#3

print('Test:', rndmtreeGS.score(test_set, test_classes), 'Entrenamiento:', rndmtreeGS.score(train_set, train_classes))

rndmtreeGS.cv_results_

- {'mean_fit_time': array([35.96888089, 34.96881421, 34.66049409, 13.40506005, 11.07682443,
         7.60593661,  4.91088072]),
 'mean_score_time': array([1.85439777, 1.7807653 , 1.85489082, 1.1338714 , 1.0313789 ,
        1.06492782, 0.93730863]),
 'mean_test_score': array([0.90792571, 0.90747571, 0.90735571, 0.81649857, 0.80138714,
        0.78183286, 0.75766   ]),
 'mean_train_score': array([0.99698571, 0.98917643, 0.98124071, 0.81721786, 0.80162   ,
        0.78222786, 0.75768786]),
 'param_min_samples_split': masked_array(data=[3, 6, 10, 0.05, 0.1, 0.3, 0.5],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_split': 3},
  {'min_samples_split': 6},
  {'min_samples_split': 10},
  {'min_samples_split': 0.05},
  {'min_samples_split': 0.1},
  {'min_samples_split': 0.3},
  {'min_samples_split': 0.5}],
 'rank_test_score': array([1, 2, 3, 4, 5, 6, 7], dtype=int32),
 'split0_test_score': array([0.90796026, 0.90807169, 0.90778455, 0.8161991 , 0.79630487,
        0.79154345, 0.75760926]),
 'split0_train_score': array([0.99701714, 0.9892707 , 0.98090926, 0.81576974, 0.79567185,
        0.78992256, 0.75607608]),
 'split1_test_score': array([0.90821701, 0.90717987, 0.90686272, 0.81468974, 0.80388972,
        0.78382826, 0.75106393]),
 'split1_train_score': array([0.99697429, 0.98908644, 0.98147287, 0.81622656, 0.80455443,
        0.78515087, 0.75143946]),
 'split2_test_score': array([0.90759987, 0.90717558, 0.90741987, 0.81860688, 0.80396686,
        0.77012681, 0.76430681]),
 'split2_train_score': array([0.99696572, 0.98917215, 0.98134001, 0.81965727, 0.80463371,
        0.77161016, 0.76554802]),
 'std_fit_time': array([0.87112457, 0.26412723, 0.32180851, 0.76698661, 0.5504719 ,
        0.17955934, 0.25931883]),
 'std_score_time': array([0.03482148, 0.01006607, 0.06159025, 0.12525036, 0.06793082,
        0.07233111, 0.07399855]),
 'std_test_score': array([0.00025313, 0.00042142, 0.00037906, 0.00161313, 0.00359386,
        0.00885643, 0.0054065 ]),
 'std_train_score': array([2.24941728e-05, 7.52856866e-05, 2.40568581e-04, 1.73497993e-03,
        4.20609807e-03, 7.75645588e-03, 5.87147012e-03])}

- Pruebo distinta cantidad min de muestras por hoja

#GridSearch, busco los mejores parametros
parameters = {'min_samples_leaf':[5, 10, 15, 0.1, 0.2, 0.3]}
rndmtree = RandomForestClassifier(n_estimators=20, min_samples_split=3, n_jobs=-1)
rndmtreeGS = GridSearchCV(rndmtree, parameters)
rndmtreeGS.fit(train_set, train_classes)
rndmtreeGS.best_params_
#5

print('Test:', rndmtreeGS.score(test_set, test_classes), 'Entrenamiento:', rndmtreeGS.score(train_set, train_classes))

rndmtreeGS.cv_results_

- {'mean_fit_time': array([34.12834422, 31.69489098, 29.53571296,  5.74226379,  4.48574471,
         4.32093422]),
 'mean_score_time': array([1.91369001, 1.71238804, 1.7388409 , 1.02940671, 0.9464554 ,
        1.048045  ]),
 'mean_test_score': array([0.90461857, 0.90086714, 0.89893143, 0.72985714, 0.70999143,
        0.63916   ]),
 'mean_train_score': array([0.95974429, 0.94108929, 0.92990714, 0.73038571, 0.71052499,
        0.63978133]),
 'param_min_samples_leaf': masked_array(data=[5, 10, 15, 0.1, 0.2, 0.3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 5},
  {'min_samples_leaf': 10},
  {'min_samples_leaf': 15},
  {'min_samples_leaf': 0.1},
  {'min_samples_leaf': 0.2},
  {'min_samples_leaf': 0.3}],
 'rank_test_score': array([1, 2, 3, 4, 5, 6], dtype=int32),
 'split0_test_score': array([0.90523456, 0.90191742, 0.89975743, 0.7312222 , 0.70183942,
        0.50290142]),
 'split0_train_score': array([0.96031423, 0.94238277, 0.93039133, 0.7304539 , 0.70249386,
        0.50290358]),
 'split1_test_score': array([0.90397843, 0.900267  , 0.898917  , 0.72487389, 0.70053529,
        0.70704101]),
 'split1_train_score': array([0.95944003, 0.94068147, 0.93065291, 0.72652877, 0.70204236,
        0.70834449]),
 'split2_test_score': array([0.90464272, 0.900417  , 0.89811985, 0.73347533, 0.72759961,
        0.70753815]),
 'split2_train_score': array([0.9594786 , 0.94020361, 0.92867719, 0.73417448, 0.72703877,
        0.70809592]),
 'std_fit_time': array([1.71880513, 0.43632298, 0.52407372, 0.12021351, 0.06704027,
        0.16466184]),
 'std_score_time': array([0.19792525, 0.15922938, 0.35129798, 0.0386923 , 0.05522493,
        0.03755701]),
 'std_test_score': array([0.00051309, 0.00074518, 0.00066862, 0.00364177, 0.01246223,
        0.09634979]),
 'std_train_score': array([0.00040332, 0.00093521, 0.00087624, 0.00312172, 0.01167846,
        0.09678724])}


- Integrando varios

#GridSearch, busco los mejores parametros
parameters = {'min_samples_leaf':[4,5,6], 'min_samples_split':[2,3,4]}
rndmtree = RandomForestClassifier(n_estimators=20, n_jobs=-1)
rndmtreeGS = GridSearchCV(rndmtree, parameters)
rndmtreeGS.fit(train_set, train_classes)
rndmtreeGS.best_params_
#{'min_samples_leaf': 4, 'min_samples_split': 3}

print('Test:', rndmtreeGS.score(test_set, test_classes), 'Entrenamiento:', rndmtreeGS.score(train_set, train_classes))

rndmtreeGS.cv_results_

{'mean_fit_time': array([34.72823882, 34.23876715, 33.93140038, 32.66352145, 33.4588023 ,
        33.58448966, 33.14943576, 33.16646902, 32.01965721]),
 'mean_score_time': array([1.94264499, 1.92731341, 3.14476267, 1.66642451, 1.76112358,
        2.63814982, 1.77258134, 1.63486902, 1.58408912]),
 'mean_test_score': array([0.90530143, 0.90588286, 0.90509571, 0.90517286, 0.90467571,
        0.90454857, 0.90399571, 0.90402   , 0.90407714]),
 'mean_train_score': array([0.96595714, 0.965945  , 0.96606214, 0.96040357, 0.95977643,
        0.95955857, 0.95472643, 0.95461929, 0.95435714]),
 'param_min_samples_leaf': masked_array(data=[4, 4, 4, 5, 5, 5, 6, 6, 6],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 4, 2, 3, 4, 2, 3, 4],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 4, 'min_samples_split': 2},
  {'min_samples_leaf': 4, 'min_samples_split': 3},
  {'min_samples_leaf': 4, 'min_samples_split': 4},
  {'min_samples_leaf': 5, 'min_samples_split': 2},
  {'min_samples_leaf': 5, 'min_samples_split': 3},
  {'min_samples_leaf': 5, 'min_samples_split': 4},
  {'min_samples_leaf': 6, 'min_samples_split': 2},
  {'min_samples_leaf': 6, 'min_samples_split': 3},
  {'min_samples_leaf': 6, 'min_samples_split': 4}],
 'rank_test_score': array([2, 1, 4, 3, 5, 6, 9, 8, 7], dtype=int32),
 'split0_test_score': array([0.90601455, 0.90652455, 0.9060017 , 0.90523027, 0.90562884,
        0.9049517 , 0.90445884, 0.9044417 , 0.90484027]),
 'split0_train_score': array([0.96583852, 0.96558781, 0.96594781, 0.96024137, 0.96033566,
        0.95990923, 0.95438279, 0.95462065, 0.95427779]),
 'split1_test_score': array([0.90470272, 0.90615129, 0.90438986, 0.90548272, 0.90399986,
        0.90459986, 0.90386701, 0.90405986, 0.90353272]),
 'split1_train_score': array([0.96589431, 0.96580217, 0.96596074, 0.96004646, 0.95931574,
        0.9594936 , 0.95525503, 0.95476432, 0.95384932]),
 'split2_test_score': array([0.90518701, 0.90497272, 0.90489558, 0.90480558, 0.90439843,
        0.90409415, 0.90366129, 0.90355843, 0.90385843]),
 'split2_train_score': array([0.9661386 , 0.96644502, 0.96627788, 0.96092289, 0.95967789,
        0.95927289, 0.95454146, 0.95447289, 0.95494432]),
 'std_fit_time': array([0.7692752 , 0.94786775, 1.47604292, 0.37104263, 1.17804001,
        0.73667547, 0.56061798, 0.73302194, 0.18785483]),
 'std_score_time': array([0.0557437 , 0.06053902, 1.67110932, 0.06282441, 0.09425761,
        1.31127325, 0.00825542, 0.06364949, 0.00286982]),
 'std_test_score': array([0.00054163, 0.00066136, 0.00067307, 0.00027941, 0.00069333,
        0.00035197, 0.00033808, 0.00036169, 0.00055576]),
 'std_train_score': array([0.00013031, 0.00036424, 0.00015264, 0.00037573, 0.00042217,
        0.00026382, 0.00037935, 0.00011898, 0.00045054])}